# Sistem penjadwalan praktikum lab otomatis menggunakan algoritma meta-heuristik hibrida berbasiskan API. Menggunakan algoritma genetika dan lainnya (dalam pengembangan).

## Setup

### Django setup and other dependencies

In [1]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()
from django.conf import settings
#set debug to true
settings.DEBUG = True

#logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Pararellization

In [2]:
#Pararellization

import os
import multiprocessing as mp
import concurrent.futures
from django.db import close_old_connections

class ParallelExecutor:
    def __init__(self, max_workers=None):
        self.max_workers = min(mp.cpu_count(), max_workers or mp.cpu_count())
        self.executor = None
        self.enter_args = None
        self.enter_kwargs = None

    def _execute_parallel(self, fn, data, cpu_bound=False):
        executor_cls = concurrent.futures.ThreadPoolExecutor if cpu_bound else concurrent.futures.ProcessPoolExecutor
        with executor_cls(max_workers=self.max_workers) as executor:
            # close_old_connections() #close old connections, this is to make sure that the database connection is closed after each request
            # return list(executor.map(fn, data))
            futures = [executor.submit(fn, d) for d in data]
            concurrent.futures.wait(futures)
            return [f.result() for f in futures]

    def execute_cpu_bound(self, fn, data):
        return self._execute_parallel(fn, data, cpu_bound=True)
    
    def execute_io_bound(self, fn, data):
        return self._execute_parallel(fn, data, cpu_bound=False)
    
    def __enter__(self, *args, **kwargs):
        self.executor = self
        self.enter_args = args
        self.enter_kwargs = kwargs
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.executor = None
        self.enter_args = None
        self.enter_kwargs = None


### Data related setup

#### Ekstraksi Data
Ekstrasi data dari model Django ke dalam objek Python.

In [3]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter
from functools import lru_cache
#Data Handling, import data from model to an object
# All random methods are for testing purposes only

class LaboratoryData:

    @classmethod
    @lru_cache(maxsize=1)
    def get_laboratories(cls):
        return Laboratory.objects.all()
    
    @classmethod
    def get_laboratory(cls, id):
        return Laboratory.objects.get(id=id)
    
    @classmethod
    def get_random_laboratory(cls):
        return Laboratory.objects.order_by('?').first()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_assistants(cls, id):
        laboratory = Laboratory.objects.get(id=id)
        if laboratory:
            return laboratory.assistants.all()
        return []
    
    @classmethod
    def get_modules(cls, id):
        laboratory = Laboratory.objects.get(id=id)
        if laboratory:
            return laboratory.modules.all()
        return []
    

class ModuleData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_modules(cls):
        return Module.objects.all()
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_module(cls, id):
        return Module.objects.get(id=id)
    
    @classmethod
    def get_random_module(cls):
        return Module.objects.order_by('?').first()
    
    @classmethod
    def get_laboratory(cls, id):
        module = Module.objects.get(id=id)
        if module:
            return module.laboratory
        return None
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_groups(cls, id):
        module = Module.objects.get(id=id)
        if module:
            return module.groups.all()
        return []
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_chapters(cls, id):
        module = Module.objects.get(id=id)
        if module:
            return module.chapters.all()
        return []
    
    @classmethod
    def get_participants(cls, id):
        module = Module.objects.get(id=id)
        if module:
            groups = module.groups.all()
            participants = []
            for group in groups:
                group_memberships = group.group_memberships.all()
                for group_membership in group_memberships:
                    participants.append(group_membership.participant)
            return participants
        return []
    
    @classmethod
    def get_assistants(cls, id):
        module = Module.objects.get(id=id)
        if module:
            assistants = []
            assistants_membership = module.assistant_memberships.all()
            for assistant_membership in assistants_membership:
                assistants.append(assistant_membership.assistant)
            return assistants
        return []
        
    
class ChapterData:
    
    @classmethod
    @lru_cache(maxsize=10)
    def get_chapters(cls):
        return Chapter.objects.all()
    
    @classmethod
    def get_chapter(cls, id):
        return Chapter.objects.get(id=id)
    
    @classmethod
    def get_module(cls, id):
        chapter = Chapter.objects.get(id=id)
        if chapter:
            return chapter.module
        return None
    
    @classmethod
    def get_random_chapter(cls, id):
        chapter = Chapter.objects.get(id=id)
        if chapter:
            return chapter.module
        return None
    
class ParticipantData:
        
    @classmethod
    @lru_cache(maxsize=10)
    def get_participants(cls):
        return Participant.objects.all()
    
    @classmethod
    def get_participant(cls, id):
        return Participant.objects.get(id=id)
    
    @classmethod
    def get_random_participant(cls):
        return Participant.objects.order_by('?').first()
    
    @classmethod
    def get_groups(cls, id):
        participant = Participant.objects.get(id=id)
        if participant:
            groups_membership = participant.group_memberships.all()
            groups = []
            for group_membership in groups_membership:
                groups.append(group_membership.group)
            return groups
        return []
    
    @classmethod
    def get_modules(cls, id):
        participant = Participant.objects.get(id=id)
        if participant:
            groups_membership = participant.group_memberships.all()
            modules = []
            for group_membership in groups_membership:
                modules.append(group_membership.group.module)
            return modules
        return []
    
    @classmethod
    def get_schedule(cls, id):
        participant = Participant.objects.get(id=id)
        if participant:
            return participant.regular_schedule
        return None
        
class GroupData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_groups(cls):
        return Group.objects.all()
    
    @classmethod
    def get_group(cls, id):
        return Group.objects.get(id=id)
    
    @classmethod
    def get_random_group(cls):
        return Group.objects.order_by('?').first()
    
    @classmethod
    def get_module(cls, id):
        group = Group.objects.get(id=id)
        if group:
            return group.module
        return None
    
    @classmethod
    def get_participants(cls, id):
        group = Group.objects.get(id=id)
        if group:
            group_memberships = group.group_memberships.all()
            participants = []
            for group_membership in group_memberships:
                participants.append(group_membership.participant)
            return participants
        return []
    
    @classmethod
    def get_assistants(cls, id):
        group = Group.objects.get(id=id)
        if group:
            module = group.module
            assistants = []
            assistants_membership = module.assistant_memberships.all()
            for assistant_membership in assistants_membership:
                assistants.append(assistant_membership.assistant)
            return assistants
        return []
    
    @classmethod
    def get_participant_schedule(cls, id):
        participants = cls.get_participants(id)
        if participants:
            schedule = []
            for participant in participants:
                schedule.append(participant.regular_schedule)
            return schedule
        return None
    
    @classmethod
    @lru_cache(maxsize=None)
    def get_schedule(cls, id):
        participants_schedule = cls.get_participant_schedule(id)
        if participants_schedule:
            days = participants_schedule[0].keys()
            merged_schedule = {day:{} for day in days}
            for day in days:
                for timeslot in participants_schedule[0][day]:
                    is_available = all([participant_schedule[day][timeslot] for participant_schedule in participants_schedule])
                    merged_schedule[day][timeslot] = is_available
            return merged_schedule
        return None
    
class AssistantData:
    
    @classmethod
    @lru_cache(maxsize=1)
    def get_assistants(cls):
        return Assistant.objects.all()
    
    @classmethod
    def get_assistant(cls, id):
        return Assistant.objects.get(id=id)
    
    @classmethod
    def get_random_assistant(cls):
        # This is for testing purposes only
        return Assistant.objects.order_by('?').first()
    
    @classmethod
    def get_laboratory(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            return assistant.laboratory
        return None
    
    @classmethod
    def get_modules(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            modules = []
            assistant_membership = assistant.assistant_memberships.all()
            for assistant_membership in assistant_membership:
                modules.append(assistant_membership.module)
            return modules
        return []
    
    @classmethod
    def get_groups(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            modules = cls.get_modules(id)
            groups = []
            for module in modules:
                groups.append(module.groups.all())
            return groups
        return []
    
    @classmethod
    def get_schedule(cls, id):
        assistant = Assistant.objects.get(id=id)
        if assistant:
            return assistant.regular_schedule
        return None


In [4]:
# import timeit
# import cProfile
# import random



# def get_group_schedule(group_id):
#     group_id = random.choice(group_id)
#     return GroupData.get_schedule(group_id)

# group_id = [group.id for group in GroupData.get_groups()]
# pr = cProfile.Profile()
# pr.enable()
# for i in range(36000):
#     get_group_schedule(group_id)
# pr.disable()

# pr.print_stats(sort='cumtime')

In [5]:
#Data manager
class DataHandler:
    def __init__(self):
        self.laboratory_data = LaboratoryData()
        self.module_data = ModuleData()
        self.chapter_data = ChapterData()
        self.participant_data = ParticipantData()
        self.group_data = GroupData()
        self.assistant_data = AssistantData()
    

##### Test

### Constants

In [6]:
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

## Algoritma

### Algoritma Genetika

#### Gene

##### Gene Inialization
Initializing the gene structure. I add constraits_broken attribute to the gene structure to keep track of the number of constraints broken by the gene, or should I rename it to fitness_broken?  So that the chromosome can detect which gene violates the most constraints and then mutate it. I don't need constraints_broken attribute because invalid genes will never be added to the chromosome.

In [7]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Gene:
    def __init__(self, laboratory, module, chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.chapter = chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot

        self._laboratory_data = None
        self._module_data = None
        self._chapter_data = None
        self._group_data = None
        self._assistant_data = None

    def __str__(self):
        return f"Gene(laboratory={self.laboratory}, module={self.module}, chapter={self.chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other: "Gene"):
        return self.laboratory == other.laboratory and self.module == other.module and self.chapter == other.chapter and self.group == other.group and self.assistant == other.assistant and self.time_slot == other.time_slot
    
    @property
    def laboratory_data(self):
        if self._laboratory_data is None:
            self._laboratory_data = LaboratoryData.get_laboratory(self.laboratory)
        return self._laboratory_data
    
    @property
    def module_data(self):
        if self._module_data is None:
            self._module_data = ModuleData.get_module(self.module)
        return self._module_data
    
    @property
    def chapter_data(self):
        if self._chapter_data is None:
            self._chapter_data = ChapterData.get_chapter(self.chapter)
        return self._chapter_data
    
    @property
    def group_data(self):
        if self._group_data is None:
            self._group_data = GroupData.get_group(self.group)
        return self._group_data
    
    @property
    def assistant_data(self):
        if self._assistant_data is None:
            self._assistant_data = AssistantData.get_assistant(self.assistant)
        return self._assistant_data
    
    @property
    def group_schedule(self):
        '''Returns the availability of the group'''
        return GroupData.get_schedule(self.group)
    
    


In [8]:
# Test Gene
gene = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id, AssistantData.get_random_assistant().id, TimeSlot("2021-01-01", "Monday", "Shift1"))
print(gene)

Gene(laboratory=1, module=1, chapter=1, group=4, assistant=2, time_slot=TimeSlot(date='2021-01-01', day='Monday', shift='Shift1'))


##### Gene Level Constraints
Check if a gene is valid by not violating any constraints. Constraints are defined on gene level, it was to ensure that the gene is valid before it is assigned to a chromosome. 

My plan is to prevent the gene from being invalid in the first place, all the constraint must return true before the gene is assigned to a chromosome. This would reduce the number of invalid chromosomes and thus reducing the complexity of the algorithm albeit make the chromosome initialization process slower, but atleast it reduces the pressure for my brain to think about fixing the invalid chromosomes. My God, if only I was a genius, I would not have to do this. I rather playing genshin impact than doing this, hu tao is waiting for me.

In [9]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

class BaseConstraint:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Constraint(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, gene: Gene):
        raise NotImplementedError("Constraint function not implemented")
    
# Ensure that each chapter is assigned to the correct module.
class ChapterModuleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ChapterModuleConstraint")
        self.chapters = ChapterData

    def __call__(self, gene: Gene) -> bool:
        if gene.module == self.chapters.get_module(gene.chapter).id:
            return True
        return False
    
# Ensure that each group is assigned to the correct module.
class GroupModuleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("GroupModuleConstraint")
        self.groups = GroupData
    
    def __call__(self, gene: Gene) -> bool:
        if gene.module == self.groups.get_module(gene.group).id:
            return True
        return False

# Ensure that each module is assigned to the correct lab.
class ModuleLaboratoryConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ModuleLaboratoryConstraint")
        self.modules = ModuleData
    
    def __call__(self, gene: Gene) -> bool:
        if gene.laboratory == self.modules.get_laboratory(gene.module).id:
            return True
        return False
    
# Ensure that each assistant is assigned to the correct lab.
class AssistantLaboratoryConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("AssistantLaboratoryConstraint")
        self.assistants = AssistantData
    
    def __call__(self, gene: Gene) -> bool:
        if gene.laboratory == self.assistants.get_laboratory(gene.assistant).id:
            return True
        return False
    
# Ensure that the group schedule is not violated by the gene time slot.
class ScheduleConstraint(BaseConstraint):
    def __init__(self):
        super().__init__("ScheduleConstraint")
        self.groups = GroupData
    
    def __call__(self, gene: Gene) -> bool:
        
        schedule = self.groups.get_schedule(gene.group)
        if schedule:
            return schedule[gene.time_slot.day][gene.time_slot.shift]
        return False
    
# Dynamic Constraint Class (custom constraint function)
class DynamicConstraint(BaseConstraint):
    def __init__(self, name, constraint_function):
        super().__init__(name)
        self.constraint_function = constraint_function
    
    def __call__(self, gene: Gene) -> bool:
        return self.constraint_function(gene)

In [10]:
# Group all constraints
from typing import List
class ConstraintManager:
    def __init__(self,constraints: List[BaseConstraint]):
        self.constraints = constraints
    
    def __str__(self):
        return f"constraints={self.constraints})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, gene: Gene) -> bool:
        return all([constraint(gene) for constraint in self.constraints])

In [11]:
# Test Constraint
chapter_module_constraint = ChapterModuleConstraint()
group_module_constraint = GroupModuleConstraint()
module_laboratory_constraint = ModuleLaboratoryConstraint()
assistant_laboratory_constraint = AssistantLaboratoryConstraint()
schedule_constraint = ScheduleConstraint()

constraint_checker = ConstraintManager([chapter_module_constraint, group_module_constraint, module_laboratory_constraint, assistant_laboratory_constraint, schedule_constraint])

gene = Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id, AssistantData.get_random_assistant().id, TimeSlot("2021-05-05", "Friday", "Shift2"))

print(constraint_checker(gene))

True


In [12]:
GroupData.get_schedule(gene.group)

{'Friday': {'Shift1': False,
  'Shift2': True,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Monday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': True},
 'Tuesday': {'Shift1': True,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': True},
 'Saturday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Thursday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Wednesday': {'Shift1': True,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': True,
  'Shift6': False}}

#### Chromosome

##### Chromosome Initialization

In [13]:
class Chromosome:
    def __init__(self, genes: List[Gene]):
        self.genes = genes
        self.fitness = None
        self.crowding_distance = None
    
    def __str__(self):
        return f"Chromosome(genes={self.genes})"
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other: "Chromosome"):
        return self.genes == other.genes
    
    def __getitem__(self, index):
        return self.genes[index]
    
    def __len__(self):
        return len(self.genes)
    
    def __iter__(self):
        return iter(self.genes)
    
    def __contains__(self, gene: Gene):
        return gene in self.genes
    
    def add_gene(self, gene: Gene):
        self.genes.append(gene)
    
    def remove_gene(self, gene: Gene):
        self.genes.remove(gene)
    
    def get_gene(self, index):
        return self.genes[index]
    
    def get_genes(self):
        return self.genes

#### Fitness Function
Fitness function is a function that takes a chromosome as input and returns a fitness score as output. The fitness score is a measure of how good the chromosome is. The higher the score, the better the chromosome is, wait, is it? Or should I say the lower the score, the better the chromosome is? I don't know, I'm just keep moving forward without any idea of what I'm actually doing, just hoping that maybe I stumble upon something that works. I'm just a monkey with a keyboard, I don't know what I'm doing.

In [14]:
#Base Fitness Class
class BaseFitness:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Fitness(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Fitness function not implemented")

##### 1. Group Assignment Conflict Penalty
This function is used to calculate the number of group that assigned to the same lab at the same time. The lower the number, the less conflict there is. In other words, this objective is to Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

Bahasa Simplenya, fungsi ini menghitung jumlah konflik yang terjadi pada jadwal, khususnya jadwal grup yang berada di lab yang sama pada waktu yang sama. Jadi misal di jadwal hari selasa shift 1, ada 2 grup yang berada di lab yang sama, maka akan dihitung sebagai 1 konflik. Tapi tergantung batas maksimal konflik yang ditentukan, misal batas maksimal konflik adalah 2, maka jadwal tersebut masih valid. Tapi kalau batas maksimal konflik adalah 1, maka jadwal tersebut tidak valid dan dihitung sebagai 1 konflik.

In [15]:
#GroupAssignmentConflictPenalty
from collections import defaultdict

class GroupAssignmentConflictFitness(BaseFitness):
    """Count the number of groups assigned to a time slot in a lab, and penalize the chromosome if the number of groups exceeds the maximum threshold"""
    def __init__(self):
        super().__init__("GroupAssignmentConflictFitness")
        self.max_threshold = 2 # Maximum number of groups that can be assigned to a single time slot in lab
        self.conflict_penalty = 1 # Penalty for each group that exceeds the maximum threshold
        
        #conflicts[laboratory][module][time_slot] = [groups]
        self._conflicts = self.default_outer_dict()
        # Keeps track of the number of groups assigned to a time slot in a lab, initialized to 0

    def default_inner_dict(self):
        return defaultdict(list)
    
    def default_middle_dict(self):
        return defaultdict(self.default_inner_dict)
    
    def default_outer_dict(self):
        return defaultdict(self.default_middle_dict)

    def __call__(self, chromosome: Chromosome):
        self._conflicts.clear()
        for gene in chromosome:
            self._conflicts[gene.laboratory][gene.module][gene.time_slot].append(gene.group)
        
        total_penalty = 0
        for laboratory in self._conflicts:
            for module in self._conflicts[laboratory]:
                for time_slot in self._conflicts[laboratory][module]:
                    groups = self._conflicts[laboratory][module][time_slot] #Get all groups that are assigned to the time slot, more specifically, the number of groups in conflict[laboratory][module][time_slot]
                    if len(groups) > self.max_threshold:
                        total_penalty += (len(groups) - self.max_threshold) * self.conflict_penalty
        return total_penalty
    
    def get_conflicts(self):
        return self._conflicts
    
    def configure(self, max_threshold, conflict_penalty):
        """Configure the fitness function
        Args:
            max_threshold (int): Maximum number of groups that can be assigned to a single time slot in lab
            conflict_penalty (int): Penalty for each group that exceeds the maximum threshold"""
        self.max_threshold = max_threshold
        self.conflict_penalty = conflict_penalty

##### 2. Assistant Distribution Penalty
The purpose of this function is to maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading, and to ensure their brain is not fried.

In [16]:
from collections import namedtuple, defaultdict, Counter

#Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
class AssistantDistributionFitness(BaseFitness):
    def __init__(self):
        super().__init__("AssistantDistributionFitness")
        self.max_group_threshold = 15 # Maximum number of groups that can be assigned to a single assistant
        self.max_shift_threshold = 15 # Maximum number of shifts that can be assigned to a single assistant
        self.group_penalty = 1 # Penalty for each group that exceeds the maximum threshold
        self.shift_penalty = 1 # Penalty for each shift that exceeds the maximum threshold

        #groups[assistant][module] = [groups]
        self._groups = self.default_outer_dict()
        #shifts[assistant][module] = [shifts]
        self._shifts = self.default_outer_dict()

    def default_inner_dict(self):
        return defaultdict(set)
    
    def default_outer_dict(self):
        return defaultdict(self.default_inner_dict)
    


    def __call__(self, chromosome: Chromosome):
        self._groups.clear()
        self._shifts.clear()
        for gene in chromosome:
            self._groups[gene.assistant][gene.module].add(gene.group)
            self._shifts[gene.assistant][gene.module].add(gene.time_slot)
        
        total_penalty = 0
        for assistant in self._groups:
            for module in self._groups[assistant]:
                groups = self._groups[assistant][module]
                shifts = self._shifts[assistant][module]
                if len(groups) > self.max_group_threshold:
                    total_penalty += (len(groups) - self.max_group_threshold) * self.group_penalty
                if len(shifts) > self.max_shift_threshold:
                    total_penalty += (len(shifts) - self.max_shift_threshold) * self.shift_penalty
        return total_penalty
    
    def get_groups(self):
        return self._groups
    
    def get_shifts(self):
        return self._shifts
    
    def configure(self, max_group_threshold, max_shift_threshold, group_penalty, shift_penalty):
        """Configure the fitness function
        Args:
            max_group_threshold (int): Maximum number of groups that can be assigned to a single assistant
            max_shift_threshold (int): Maximum number of shifts that can be assigned to a single assistant
            group_penalty (int): Penalty for each group that exceeds the maximum threshold
            shift_penalty (int): Penalty for each shift that exceeds the maximum threshold"""
        self.max_group_threshold = max_group_threshold
        self.max_shift_threshold = max_shift_threshold
        self.group_penalty = group_penalty
        self.shift_penalty = shift_penalty


##### 3. Apalagi ya? 
Nanti ditambahin lagi kalo udah kepikiran.


In [17]:
#Fitness Manager
class FitnessManager:
    '''FitnessManager is a class that manages all fitness functions and their respective fitness values'''
    def __init__(self, fitness_functions: List[BaseFitness]):
        self.fitness_functions = fitness_functions
    
    def __str__(self):
        return f"FitnessManager(fitness_functions={self.fitness_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        return sum([fitness_function(chromosome) for fitness_function in self.fitness_functions])
    
    def configure(self, fitness_functions: List[BaseFitness]):
        """Configure the fitness manager
        Args:
            fitness_functions (List[BaseFitness]): List of fitness functions"""
        
        self.fitness_functions = fitness_functions

    def grouped_fitness(self, chromosome: Chromosome):
        """Return a dictionary of fitness functions and their respective fitness value"""
        return {fitness_function.name: fitness_function(chromosome) for fitness_function in self.fitness_functions}

#### Population

##### Population Initialization

In [18]:
#Population Initialization Class
import random

class Population:
    def __init__(self, chromosomes: List[Chromosome], fitness_manager: FitnessManager):
        self.chromosomes = chromosomes
        self.fitness_manager = fitness_manager
    
    def __str__(self):
        return f"Population(chromosomes={self.chromosomes})"
    
    def __repr__(self):
        return self.__str__()
    
    def __getitem__(self, index):
        return self.chromosomes[index]
    
    def __len__(self):
        return len(self.chromosomes)
    
    def __iter__(self):
        return iter(self.chromosomes)
    
    def __eq__(self, other: "Population"):
        return self.chromosomes == other.chromosomes
    
    def __contains__(self, chromosome: Chromosome):
        return chromosome in self.chromosomes
    
    def calculate_fitness(self):
        for chromosome in self.chromosomes:
            chromosome.fitness = self.fitness_manager(chromosome)
    
    def add_chromosome(self, chromosome: Chromosome):
        self.chromosomes.append(chromosome)
    
    def remove_chromosome(self, chromosome: Chromosome):
        self.chromosomes.remove(chromosome)
    
    def get_chromosome(self, index):
        return self.chromosomes[index]
    
    def get_chromosomes(self):
        return self.chromosomes
    
    def get_random_chromosome(self):
        return random.choice(self.chromosomes)
    
    def get_best_chromosome(self):
        return min(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def get_worst_chromosome(self):
        return max(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def get_average_fitness(self):
        return sum([chromosome.fitness for chromosome in self.chromosomes]) / len(self.chromosomes)

#### Factory Method
Class for generating population, chromosome, and gene. This class also generates some data like time slot, chromosome length, etc.This class requires a lot of data to be passed to it, so I think it's better to make it a class instead of a function.

In [19]:
#factory.py
from typing import List
from math import ceil, floor

import numpy as np
from datetime import datetime, timedelta

from multiprocessing import Pool

TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Factory:
    '''Factory class to generate chromosomes and population. It also contains the data from the database. We can call this class when we need some of'''
    def __init__(self):
        self.laboratories = LaboratoryData
        self.modules = ModuleData
        self.chapters = ChapterData
        self.groups = GroupData
        self.participants = ParticipantData
        self.assistants = AssistantData
        self.constant = Constant
        
        self.constraints = ConstraintManager([ChapterModuleConstraint(), GroupModuleConstraint(), ModuleLaboratoryConstraint(), AssistantLaboratoryConstraint(), ScheduleConstraint()])
        self.fitness_manager = FitnessManager([GroupAssignmentConflictFitness(), AssistantDistributionFitness()])

    def set_constraints(self, constraints: List[BaseConstraint]):
        # No idea if I need to check constraints here when generating the population
        # But I think it's better to set the constraints here, maybe we can change the constraints later
        self.constraints = ConstraintManager(constraints)

    def set_fitness_manager(self, fitness_manager: FitnessManager):
        self.fitness_manager = fitness_manager

    def set_data(self, laboratories, modules, chapters, groups, participants, assistants):
        self.laboratories = laboratories
        self.modules = modules
        self.chapters = chapters
        self.groups = groups
        self.participants = participants
        self.assistants = assistants
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7) # Number of weeks between start date and end date, using floor to make sure that the time slot does not exceed the end date
        random_weeks = np.random.randint(0, weeks_duration)
        random_days = np.random.choice(self.constant.days)
        random_shifts = np.random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + self.constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)
    
    def generate_time_slot_weekly(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        random_days = np.random.choice(self.constant.days)
        random_shifts = np.random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days= 7 + self.constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)
    
    def chromosome_size(self) -> int:
        """
        Calculate the chromosome size based on the number of modules chapters and groups.
        Each group must be assigned to all chapters in a module. Mostly used for testing purposes.
        The actual chromosome size is generated based on parential data.

        Returns:
            int: chromosome size
        """
        chromosome_size = 0
        for module in self.modules:
            chromosome_size += len(module.groups.all()) * len(module.chapters.all())
        return chromosome_size
    
    def generate_chromosome(self) -> Chromosome:
        """Generate a chromosome based on data, each group must be assigned to all chapters in a module of appropriate lab"""
        chromosome = Chromosome([])
        for module in self.modules.get_modules():
            for group in self.modules.get_groups(module.id):
                for chapter in self.modules.get_chapters(module.id):
                    laboratory = module.laboratory
                    assistant = np.random.choice(self.laboratories.get_assistants(laboratory.id))
                    time_slot = self.generate_time_slot(module.start_date, module.end_date)
                    gene = Gene(laboratory.id, module.id, chapter.id, group.id, assistant.id, time_slot)
                    chromosome.add_gene(gene)
        return chromosome
    
    def generate_chromosome_weekly(self) -> Chromosome:
        """Generate a chromosome based on data, each group must be assigned to all chapters in a module of appropriate lab"""
        chromosome = Chromosome([])
        for module in self.modules.get_modules():
            for group in self.modules.get_groups(module.id):
                start_date = module.start_date
                end_date = module.end_date
                duration = (end_date - start_date).days + 1
                weeks_duration = floor(duration / 7)
                chapters_count = len(self.modules.get_chapters(module.id))
                weekly_chapters = ceil(chapters_count / weeks_duration)
                for i in range(weekly_chapters):
                    laboratory = module.laboratory
                    assistant = np.random.choice(self.laboratories.get_assistants(laboratory.id))
                    time_slot = self.generate_time_slot_weekly(start_date, end_date)
                    #first chapter
                    chapter = module.chapters.all().first()
                    gene = Gene(laboratory.id, module.id, chapter.id, group.id, assistant.id, time_slot)
                    chromosome.add_gene(gene)
        return chromosome
    
    def generate_population(self, population_size: int, fitness_manager: FitnessManager = None, weekly = False) -> Population:
        """Generate a population based on the population size"""

        if settings.DEBUG:
            logger.info("Generating population")

        if fitness_manager:
            self.fitness_manager = fitness_manager

        if weekly:
            return Population([self.generate_chromosome_weekly() for _ in range(population_size)], self.fitness_manager)
        else:
            return Population([self.generate_chromosome() for _ in range(population_size)], self.fitness_manager)
            
        # chromosomes = [generator() for _ in range(population_size)]
        # population = Population(chromosomes, self.fitness_manager)
        # population.calculate_fitness()
        # return population
    
    # def generate_population_weekly(self, population_size: int) -> Population:
    #     """Generate a population based on the population size"""

    #     if settings.DEBUG:
    #         logger.info("Generating population")

    #     chromosomes = []
    #     # chromosomes = [self.generate_chromosome() for i in range(population_size)]
    #     for i in range(population_size):
    #         if settings.DEBUG:
    #             logger.info(f"Generating chromosome {i}")

    #         chromosomes.append(self.generate_chromosome_weekly())

    #     population = Population(chromosomes, self.fitness_manager)
    #     #population.calculate_fitness()
    #     return population
    


##### Factory pararel version test

In [20]:
#Factory class parallel version test for faster population generation

from multiprocessing import Pool, cpu_count, current_process, Manager, Process, Queue, Lock, Value
from functools import partial
from itertools import repeat
import concurrent.futures

class FactoryParallel(Factory):
    def __init__(self):
        super().__init__()
        self.pool = Pool()
        
    def __del__(self):
        self.pool.close()
        self.pool.join()
    
    def generate_chromosome_parallel(self, i) -> Chromosome:
        # i is just a dummy variable, it is not used, but it is needed for the map function
        return self.generate_chromosome()
    
    def generate_population_parallel(self, population_size: int) -> Population:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            chromosomes = list(executor.map(self.generate_chromosome_parallel, range(population_size)))
        population = Population(chromosomes, self.fitness_manager)
        return population


In [21]:
# #generate population test
# factory = Factory()
# factory_parallel = FactoryParallel()
# #time
# import time
# start = time.time()
# population = factory.generate_population(100)
# end = time.time()

# parallel_start = time.time()
# population_parallel = factory_parallel.generate_population_parallel(100)
# parallel_end = time.time()

# print(f"Time taken for serial population generation: {end - start}")
# print(f"Time taken for parallel population generation: {parallel_end - parallel_start}")


In [22]:
import timeit

def test_factory(population_size):
    factory_serial = Factory()
    factory_parallel = FactoryParallel()
    serial_time = timeit.timeit(lambda: factory_serial.generate_population(population_size), number=1)
    parallel_time = timeit.timeit(lambda: factory_parallel.generate_population_parallel(population_size), number=1)
    print(f"Population size: {population_size}")
    print(f"Serial time: {serial_time}")
    print(f"Parallel time: {parallel_time}")
    print(f"Speedup: {serial_time / parallel_time}")
    if serial_time / parallel_time > 1:
        print("Parallel is faster with time difference of: ", serial_time - parallel_time)
    else:
        print("Serial is faster with time difference of: ", parallel_time - serial_time)
    print()
    return serial_time, parallel_time


In [23]:
# #test for population size 1, 2, 4, 8, 16, 32, 64, 128, 256, 512
# settings.DEBUG = False
# data = []
# for i in range(0, 10):
#     data.append(test_factory(2 ** i))
#     print('----------------------------------')


##### Pararel and Serial version comparison

Setelah dicoba, pada awalnya versi pararel lebih lambat dari versi serial, khususnya pada ukuran populasi rendah. Sekiranya antara 1 hingga 100 populasi, versi serial lebih cepat dari pada yang pararel. Tapi ketika ukuran populasi meningkat, setidaknya diatas 100, kecepatan versi pararel mulai mengungguli versi serial. Perbedaan yang dihasilkan juga cukup signifikan, pada saat populasi mencapai 512, versi pararel 1.5 kali lebih cepat dari versi serial. Apakah itu cukup signifikan? Mungkin kita coba lagi dengan ukuran populasi yang lebih besar, misal 1024 atau 2048. Sementara itu, ini adalah hasil perbandingan dari ujicoba sebelumnya. Untuk keterangan, ukuran chromosome untuk tiap populasi adalah 288.

| Population Size | Serial | Pararel | Pararel Speedup |
| --------------- | ------ | ------- | --------------- |
| 2               | 0.765  | 0.576   | 1.328           |
| 4               | 1.792  | 1.954   | 0.917           |
| 8               | 3.573  | 4.294   | 0.832           |
| 16              | 6.771  | 8.302   | 0.815           |
| 32              | 13.199 | 17.507  | 0.753           |
| 64              | 28.468 | 34.396  | 0.827           |
| 128             | 53.476 | 46.242  | 1.156           |
| 256             | 91.285 | 83.171  | 1.097           |
| 512             | 466.15 | 294.25  | 1.584           |
| 1024            | 385.61 | 304.31  | 1.267           |
| 2048            | 758.55 | 587.89  | 1.290           |

Kayaknya nanti bisa dipertimbangin deh mau make jenis proses yang mana, tergantung jumlah populasinya. Hmm kira2 bisa ngga ini diterapin di fungsi fitness. Di versi kode sebelumnya sebenernya lebih lama di ngurusin perhitungan fitness-nya, kalo generate population mah cuman diawal doang.

In [24]:
# # #test for population size 1024, 2048, 4096, 8192
# for i in range(10, 14):
#     data.append(test_factory(2 ** i))
#     print('----------------------------------')

#### Operator
Operator is a class that contains methods for performing genetic operations such as crossover and mutation. This class is used by the genetic algorithm class to perform genetic operations on the population.

##### Mutation

The one that need to mutate is just the time slot and the assistant, the rest is fixed because we just need the schedule. 
The group is already assigned to the appropriate module and all its chapters, we don't need to change it. 
Each chapter also already assigned to the correct module, and each module have their own lab, as well the group and the assistant. 
The schedule is affected by the time slot and the assistant that is assigned to the group. 
The number of group is fixed and cannot be random, that's why the only things that need to be mutated is the time slot and the assistant only.

Tapi masih rada bingung sih fungsi mutasi apa aja yang mesti diterapin, apa ini bener?

In [25]:
#Mutation Class

import random
from copy import deepcopy

class BaseMutation:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the mutation function being called if more than one mutation function is used.
    
    def __str__(self):
        return f"Mutation(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Mutation function not implemented")
    
class SwapMutation(BaseMutation):
    def __init__(self):
        super().__init__("SwapMutation")
    
    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene1 = random.choice(chromosome)
        # Randomly select another gene
        gene2 = random.choice(chromosome)
        # Swap the time slot and the assistant
        gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
        gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant

        return chromosome
    
class ShiftMutation(BaseMutation):
    def __init__(self):
        super().__init__("ShiftMutation")
        self.constant = Constant
    
    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene = random.choice(chromosome)
        # Shift the time slot
        gene.time_slot = self.shift_time_slot(gene.time_slot)

        return chromosome
    
    def shift_time_slot(self, time_slot: TimeSlot) -> TimeSlot:
        # Shift the time slot by 1 day
        if time_slot.day == "Saturday":
            return TimeSlot(time_slot.date + timedelta(days=2), "Monday", time_slot.shift)
        return TimeSlot(time_slot.date + timedelta(days=1), self.constant.days[self.constant.days.index(time_slot.day) + 1], time_slot.shift)
    
class RandomMutation(BaseMutation):
    def __init__(self):
        super().__init__("RandomMutation")
        self.constant = Constant
        self.laboratories = LaboratoryData


    def __call__(self, chromosome: Chromosome):
        # Randomly select a gene
        gene = random.choice(chromosome)
        module = gene.module_data
        # Randomly select a time slot
        time_slot = self.generate_time_slot(module.start_date, module.end_date)
        # Randomly select an assistant
        assistant = random.choice(self.laboratories.get_assistants(gene.laboratory)).id 
        # Change the gene
        gene.time_slot = time_slot
        gene.assistant = assistant

        return chromosome
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7)
        random_weeks = np.random.randint(0, weeks_duration)
        random_days = np.random.choice(self.constant.days)
        random_shifts = np.random.choice(self.constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + self.constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)
    
class DynamicMutation(BaseMutation):
    def __init__(self, name, mutation_function):
        super().__init__(name)
        self.mutation_function = mutation_function
    
    def __call__(self, chromosome: Chromosome):

        return self.mutation_function(chromosome)
    
class MutationManager:
    def __init__(self, mutation_functions: List[BaseMutation]):
        self.mutation_functions = mutation_functions
        self.mutation_probability = 0.1
    
    def __str__(self):
        return f"MutationManager(mutation_functions={self.mutation_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        #random based on probability weight
        if random.random() < self.mutation_probability:
            if settings.DEBUG:
                logger.info("Mutating chromosome")
            mutation_function = self.get_random_mutation()
            return mutation_function(chromosome)
        return chromosome
    
    def get_random_mutation(self):
        return random.choices(self.mutation_functions, weights=[mutation.probability_weight for mutation in self.mutation_functions])[0]
    
    def configure(self, mutation_functions: List[BaseMutation]):
        self.mutation_functions = mutation_functions

In [26]:
# factory = Factory()
# population = factory.generate_population(5)

In [27]:
# # Mutation Test
# settings.DEBUG = True

# swap_mutation = SwapMutation()
# shift_mutation = ShiftMutation()
# random_mutation = RandomMutation()

# random_mutation.probability_weight = 1000

# mutation_manager = MutationManager([swap_mutation, shift_mutation, random_mutation])
# mutation_manager.mutation_probability = 1

# import timeit
# timeit.timeit(lambda: mutation_manager(population[0]), number=1)


In [28]:
#mutation unit test
import unittest

class MutationTest(unittest.TestCase):
    def setUp(self):
        self.factory = Factory()
        self.population = self.factory.generate_population(5)
        self.mutation_manager = MutationManager([SwapMutation(), ShiftMutation(), RandomMutation()])
        self.mutation_manager.mutation_probability = 1
        self.mutation_manager.configure([SwapMutation(), ShiftMutation(), RandomMutation()])
    
    def test_swap_mutation(self):
        swap_mutation = SwapMutation()
        chromosome = deepcopy(self.population[0])
        mutated_chromosome = swap_mutation(chromosome)
        self.assertNotEqual(chromosome, mutated_chromosome)
    
    def test_shift_mutation(self):
        shift_mutation = ShiftMutation()
        chromosome = deepcopy(self.population[0])
        mutated_chromosome = shift_mutation(chromosome)
        self.assertNotEqual(chromosome, mutated_chromosome)
    
    def test_random_mutation(self):
        random_mutation = RandomMutation()
        chromosome = deepcopy(self.population[0])
        mutated_chromosome = random_mutation(chromosome)
        self.assertNotEqual(chromosome, mutated_chromosome)
    
    def test_mutation_manager(self):
        chromosome = deepcopy(self.population[0])
        mutated_chromosome = self.mutation_manager(chromosome)
        self.assertNotEqual(chromosome, mutated_chromosome)
    
    def test_dynamic_mutation(self):
        dynamic_mutation = DynamicMutation("DynamicMutation", lambda chromosome: chromosome)
        chromosome = deepcopy(self.population[0])
        mutated_chromosome = dynamic_mutation(chromosome)
        self.assertEqual(chromosome, mutated_chromosome)

##### Crossover

In [29]:
#Crossover Class

import random
from copy import deepcopy

class BaseCrossover:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the crossover function being called if more than one crossover function is used.
    
    def __str__(self):
        return f"Crossover(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        raise NotImplementedError("Crossover function not implemented")
    
class SinglePointCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("SinglePointCrossover")
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # Randomly select a point
        point = random.randint(0, len(parent1))
        # initialize the children
        # child1 = deepcopy(parent1)
        # child2 = deepcopy(parent2)
        child1 = parent1 # already deepcopy in the selection part of the algorithm
        child2 = parent2
        #swap that point onwards
        child1.genes[point:], child2.genes[point:] = child2.genes[point:], child1.genes[point:]

        return child1, child2
    
class TwoPointCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("TwoPointCrossover")
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # Randomly select 2 points
        point1 = random.randint(0, len(parent1))
        point2 = random.randint(0, len(parent1))
        # initialize the children
        child1 = parent1
        child2 = parent2
        #swap a section of the chromosome between the 2 points
        if point1 > point2:
            point1, point2 = point2, point1
        child1.genes[point1:point2], child2.genes[point1:point2] = child2.genes[point1:point2], child1.genes[point1:point2]

        return child1, child2
    
class UniformCrossover(BaseCrossover):
    def __init__(self):
        super().__init__("UniformCrossover")
        self.uniform_probability = 0.5
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        # initialize the children
        child1 = parent1
        child2 = parent2
        #swap a section of the chromosome between the 2 points
        count = 0
        for i in range(len(child1)):
            if random.random() < self.uniform_probability:
                count += 1
                child1.genes[i], child2.genes[i] = child2.genes[i], child1.genes[i]
        
        return child1, child2
    
    def configure(self, uniform_probability):
        '''Configure the crossover function
        
        Args:
            uniform_probability (float): The probability of swapping a gene between the 2 parents on a particular index'''
        
        self.uniform_probability = uniform_probability

class DynamicCrossover(BaseCrossover):
    def __init__(self, name, crossover_function):
        super().__init__(name)
        self.crossover_function = crossover_function
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        return self.crossover_function(parent1, parent2)
    
class CrossoverManager:
    def __init__(self, crossover_functions: List[BaseCrossover]):
        self.crossover_functions = crossover_functions
        self.crossover_probability = 0.1
    
    def __str__(self):
        return f"CrossoverManager(crossover_functions={self.crossover_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, parent1: Chromosome, parent2: Chromosome):
        #random based on probability weight
        if random.random() < self.crossover_probability:
            if settings.DEBUG:
                logger.info("Crossovering chromosome")
            crossover_function = self.get_random_crossover()
            return crossover_function(parent1, parent2)
        return parent1, parent2
    
    def get_random_crossover(self):
        return random.choices(self.crossover_functions, weights=[crossover.probability_weight for crossover in self.crossover_functions])[0]
    
    def configure(self, crossover_functions: List[BaseCrossover]):
        self.crossover_functions = crossover_functions

In [30]:
# Crossover Test if it works
import unittest

#import Chromosome, Gene, TimeSlot, LaboratoryData, ModuleData, ChapterData, GroupData, AssistantData
#import SinglePointCrossover, TwoPointCrossover, UniformCrossover
#import Constant

class TestCrossover(unittest.TestCase):
    def setUp(self):
        self.parent1 = Chromosome([Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id, AssistantData.get_random_assistant().id, self.generate_time_slot(datetime(2021, 5, 5), datetime(2021, 6, 15))) for i in range(10)])
        self.parent2 = Chromosome([Gene(LaboratoryData.get_random_laboratory().id, ModuleData.get_random_module().id, ChapterData.get_random_chapter(1).id, GroupData.get_random_group().id, AssistantData.get_random_assistant().id, self.generate_time_slot(datetime(2021, 5, 5), datetime(2021, 6, 15))) for i in range(10)])

    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7)
        random_weeks = np.random.randint(0, weeks_duration)
        random_days = np.random.choice(Constant.days)
        random_shifts = np.random.choice(Constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + Constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)

    def test_single_point_crossover(self):
        single_point_crossover = SinglePointCrossover()
        child1, child2 = single_point_crossover(self.parent1, self.parent2)
        self.assertNotEqual(self.parent1.genes, child1.genes)
        self.assertNotEqual(self.parent2.genes, child2.genes)
        self.assertEqual(len(child1), len(child2))
        self.assertEqual(len(child1), len(self.parent1))
        self.assertEqual(len(child2), len(self.parent2))

    def test_two_point_crossover(self):
        two_point_crossover = TwoPointCrossover()
        child1, child2 = two_point_crossover(self.parent1, self.parent2)
        self.assertNotEqual(self.parent1.genes, child1.genes)
        self.assertNotEqual(self.parent2.genes, child2.genes)
        self.assertEqual(len(child1), len(child2))
        self.assertEqual(len(child1), len(self.parent1))
        self.assertEqual(len(child2), len(self.parent2))

    def test_uniform_crossover(self):
        uniform_crossover = UniformCrossover()
        child1, child2 = uniform_crossover(self.parent1, self.parent2)
        self.assertNotEqual(self.parent1.genes, child1.genes)
        self.assertNotEqual(self.parent2.genes, child2.genes)
        self.assertEqual(len(child1), len(child2))
        self.assertEqual(len(child1), len(self.parent1))
        self.assertEqual(len(child2), len(self.parent2))


In [31]:
# # Crossover Test
# settings.DEBUG = True
# factory = Factory()
# population = factory.generate_population(2)

# single_point_crossover = SinglePointCrossover()
# two_point_crossover = TwoPointCrossover()
# uniform_crossover = UniformCrossover()

# crossover_manager = CrossoverManager([single_point_crossover, two_point_crossover, uniform_crossover])
# crossover_manager.crossover_probability = 1

# child1, child2 = crossover_manager(population[0], population[1])

In [32]:
# # count the number of genes that are different
# count = 0
# for i in range(len(child1)):
#     if child1[i] != population[0][i]:
#         count += 1

# print(f"Number of genes that are different: {count}")

##### Repair

In [33]:
#Repairs Class

class BaseRepair:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Repair(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Repair function not implemented")
    
class DynamicRepair(BaseRepair):
    def __init__(self, name, repair_function):
        super().__init__(name)
        self.repair_function = repair_function
    
    def __call__(self, chromosome: Chromosome):
        return self.repair_function(chromosome)
    
class RepairManager:
    def __init__(self, repair_functions: List[BaseRepair]):
        self.repair_functions = repair_functions
    
    def __str__(self):
        return f"RepairManager(repair_functions={self.repair_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        for repair_function in self.repair_functions:
            chromosome = repair_function(chromosome)
        return chromosome
    
    def configure(self, repair_functions: List[BaseRepair]):
        self.repair_functions = repair_functions

In [34]:

class RepairTimeSlot(BaseRepair):
    def __init__(self):
        super().__init__("RepairTimeSlot")
        self.schedule_constraint = ScheduleConstraint()
    
    def __call__(self, chromosome: Chromosome):
        # chromosome = deepcopy(chromosome)
        for gene in chromosome:
            if not self.check_available_time_slot(gene):
                time_slot = self.find_feasible_solution(gene)
                if time_slot is None:
                    time_slot = gene.time_slot
                gene.time_slot = time_slot
        return chromosome
    
    def check_available_time_slot(self, gene: Gene):
        """Check whether the time slot is available or not.
        The gene contain group_schedule property which is a dictionary of day as key and list of shifts dictionary with boolean value as value.
        This is the example of the group_schedule property:
        {'Friday': {'Shift1': False,
                    'Shift2': True,
                    'Shift3': False,
                    'Shift4': True,
                    'Shift5': False,
                    'Shift6': True}
        }

        The value is already in a boolean value, so we can just return it based on the time slot shift and day.
        """
        return self.schedule_constraint(gene)
    
    def generate_time_slot(self, start_date, end_date):
        """Generate time slots based on the start date, end date, days and shifts"""
        #if start_date not start from Monday, then start from the next Monday
        if start_date.weekday() != 0:
            start_date = start_date + timedelta(days=7 - start_date.weekday())
        duration = (end_date - start_date).days + 1
        weeks_duration = floor(duration / 7)
        random_weeks = np.random.randint(0, weeks_duration)
        random_days = np.random.choice(Constant.days)
        random_shifts = np.random.choice(Constant.shifts)
        random_date = start_date + timedelta(days=random_weeks * 7 + Constant.days.index(random_days))
        return TimeSlot(random_date, random_days, random_shifts)
    
    def find_feasible_solution(self, gene: Gene, max_iteration=100):
        """Find a feasible solution for the gene by randomly generating a time slot until it is available"""
        start_date = gene.module_data.start_date
        end_date = gene.module_data.end_date
        for _ in range(max_iteration):
            gene.time_slot = self.generate_time_slot(start_date, end_date)
            if self.check_available_time_slot(gene):
                return gene.time_slot
        return None

In [35]:
# #test def generate_time_slot
# factory = Factory()
# repair_time_slot = RepairTimeSlot()
# population = factory.generate_population(2042)

# def test_repair_time_slot():
#     for chromosome in population:
#         chromosome = repair_time_slot(chromosome)

In [36]:
#unit test for repair time slot
import unittest

class RepairTimeSlotTest(unittest.TestCase):
    def setUp(self):
        self.factory = Factory()
        self.repair_time_slot = RepairTimeSlot()
        self.population = self.factory.generate_population(10)
    
    def test_repair_time_slot(self):
        for chromosome in self.population:
            chromosome = self.repair_time_slot(chromosome)
            for gene in chromosome:
                self.assertTrue(self.repair_time_slot.check_available_time_slot(gene))

In [37]:
# settings.DEBUG = False

# factory = Factory()
# population = factory.generate_population(500)

# repair_time_slot = RepairTimeSlot()

# repair_manager = RepairManager([repair_time_slot])

# schedule_constraint = ScheduleConstraint()

# for chromosome in population:
#     chromosome = repair_manager(chromosome)
#     count = 0
#     for gene in chromosome:
#         if not schedule_constraint(gene):
#             count += 1
#     print(f"Number of genes that violate the schedule constraint: {count}")
    

##### Selection

In [38]:
#Selection Class

import random
from copy import deepcopy

class BaseSelection:
    def __init__(self, name, probability_weight=1):
        self.name = name
        self.probability_weight = probability_weight # It is used to determine the probability of the selection function being called if more than one selection function is used.
    
    def __str__(self):
        return f"Selection(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, population: Population):
        raise NotImplementedError("Selection function not implemented")
        
    
class RouletteWheelSelection(BaseSelection):
    def __init__(self):
        super().__init__("RouletteWheelSelection")
    
    def __call__(self, population: Population):
        # Calculate the total fitness
        total_fitness = sum([chromosome.fitness for chromosome in population])
        # Calculate the probability of each chromosome being selected
        probabilities = [chromosome.fitness / total_fitness for chromosome in population]
        # Select a chromosome based on the probability

        return random.choices(population, weights=probabilities)[0] # random.choices return a list, so we need to get the first element
    
class TournamentSelection(BaseSelection):
    def __init__(self):
        super().__init__("TournamentSelection")
        self.tournament_size = 2
    
    def __call__(self, population: Population):
        # Select a random chromosome
        chromosome = random.choice(population)
        # Select a random chromosome from the tournament size
        for i in range(self.tournament_size - 1):
            chromosome2 = random.choice(population)
            if chromosome2.fitness < chromosome.fitness:
                chromosome = chromosome2 # Select the chromosome with the lowest fitness

        return chromosome
    
    def configure(self, tournament_size):
        self.tournament_size = tournament_size

class ElitismSelection(BaseSelection):
    def __init__(self):
        super().__init__("ElitismSelection")
        self.elitism_size = 1
    
    def __call__(self, population: Population):
        # Sort the population based on fitness
        population = sorted(population, key=lambda chromosome: chromosome.fitness)
        # Select the best chromosome

        if self.elitism_size == 1:
            return population[0]
        
        return population[:self.elitism_size]
    
    def configure(self, elitism_size):
        self.elitism_size = elitism_size

class DynamicSelection(BaseSelection):
    def __init__(self, name, selection_function):
        super().__init__(name)
        self.selection_function = selection_function
    
    def __call__(self, population: Population):
        return self.selection_function(population)
    
class SelectionManager:
    def __init__(self, selection_functions: List[BaseSelection]):
        self.selection_functions = selection_functions
        self.selection_probability = 0.1
    
    def __str__(self):
        return f"SelectionManager(selection_functions={self.selection_functions})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, population: Population) -> Chromosome:
        #random based on probability weight
        if random.random() < self.selection_probability:
            if settings.DEBUG:
                logger.info("Selecting chromosome")
            selection_function = self.get_random_selection()
            return selection_function(population)
        
        # if isinstance(population, list):
        #     return random.choice(population)

        return population.get_random_chromosome()
    
    def get_random_selection(self):
        return random.choices(self.selection_functions, weights=[selection.probability_weight for selection in self.selection_functions])[0]
    
    def configure(self, selection_functions: List[BaseSelection]):
        self.selection_functions = selection_functions

In [39]:
# Selection Test
factory = Factory()
population = factory.generate_population(5)

roulette_wheel_selection = RouletteWheelSelection()
tournament_selection = TournamentSelection()
elitism_selection = ElitismSelection()

selection_manager = SelectionManager([roulette_wheel_selection, tournament_selection, elitism_selection])
selection_manager.selection_probability = 1


INFO:__main__:Generating population


In [40]:
population.calculate_fitness()

#### Main Algorithm

In [41]:
#Genetic Algorithm Class

import time
from copy import deepcopy

class GeneticAlgorithm:
    def __init__(self, factory: Factory, population_size: int, fitness_manager: FitnessManager, selection_manager: SelectionManager, crossover_manager: CrossoverManager, mutation_manager: MutationManager, repair_manager: RepairManager, elitism_size: int, elitism_selection: ElitismSelection):
        self.factory = factory
        self.population_size = population_size
        self.fitness_manager = fitness_manager
        self.selection_manager = selection_manager
        self.crossover_manager = crossover_manager
        self.mutation_manager = mutation_manager
        self.repair_manager = repair_manager
        self.elitism_size = elitism_size
        self.elitism_selection = elitism_selection

        #log detail for each iteration. Containt the best chromosome, total repairing time, total crossovering time, total mutating time, total fitness time etc.
        # log[i] = { "best_chromosome": Chromosome, "repair_time": float, "crossover_time": float, "mutation_time": float, "fitness_time": float, "total_time": float}
        self.log = {}

    # def log_detail(self, i, best_chromosome, repair_time, crossover_time, mutation_time, fitness_time, total_time):

    #     self.log[i] = { "best_chromosome": best_chromosome, "repair_time": repair_time, "crossover_time": crossover_time, "mutation_time": mutation_time, "fitness_time": fitness_time, "total_time": total_time}
    
    def __str__(self):
        return f"GeneticAlgorithm(factory={self.factory}, population_size={self.population_size}, selection_manager={self.selection_manager}, crossover_manager={self.crossover_manager}, mutation_manager={self.mutation_manager}, repair_manager={self.repair_manager}, elitism_size={self.elitism_size})"
    
    def __repr__(self):
        return self.__str__()
    
    def __init_population(self):
        #return self.factory.generate_population_weekly(self.population_size)
        return self.factory.generate_population(self.population_size, self.fitness_manager)
    
    def __selection(self, population: Population):
        return self.selection_manager(population)
    
    def __crossover(self, parent1: Chromosome, parent2: Chromosome):
        return self.crossover_manager(parent1, parent2)
    
    def __mutation(self, chromosome: Chromosome):
        return self.mutation_manager(chromosome)
    
    def __repair(self, chromosome: Chromosome):
        return self.repair_manager(chromosome)
    
    def __elitism(self, population: Population):
        self.elitism_selection.elitism_size = self.elitism_size
        return self.elitism_selection(population)
    
    def __evolve(self, population: Population, iteration: int):
        start = time.time()

        # Selection
        parent1 = deepcopy(self.__selection(population))
        parent2 = deepcopy(self.__selection(population))
        selection_time = time.time() - start
        self.log[iteration]["selection_time"] += selection_time

        # Crossover
        child1, child2 = self.__crossover(parent1, parent2) # Crossover is done using deep copy, so we need to assign it back to the variable since it not modify the original chromosome
        crossover_time = time.time() - start - selection_time
        self.log[iteration]["crossover_time"] += crossover_time

        # Mutation
        self.__mutation(child1) # Mutation is done in place, so we don't need to assign it back to the variable
        self.__mutation(child2)
        mutation_time = time.time() - start - selection_time - crossover_time
        self.log[iteration]["mutation_time"] += mutation_time

        # Repair
        self.__repair(child1) # Repair is done in place, so we don't need to assign it back to the variable
        self.__repair(child2)
        repair_time = time.time() - start - selection_time - crossover_time - mutation_time
        self.log[iteration]["repair_time"] += repair_time
        
        return child1, child2
    
    def __evolve_population(self, population: Population, iteration: int):
    
        # Elitism
        start = time.time()
        elitism = self.__elitism(population)
        elitism_time = time.time() - start
        self.log[iteration]["elitism_time"] += elitism_time
        # Evolve the rest of the population
        children = []

        # for _ in range(len(population) - self.elitism_size): # -elitism_size because we need to make slots for the elitism
        #     child1, child2 = self.__evolve(population, iteration)
        #     children.append(child1)
        #     children.append(child2)

        while len(children) < len(population) - self.elitism_size:
            child1, child2 = self.__evolve(population, iteration)
            children.append(child1)
            children.append(child2)
            
        return Population(children, population.fitness_manager), elitism
    
    def run(self, max_iteration: int):
        # Initialize the population
        population = self.__init_population()
        # Calculate the fitness of the population
        population.calculate_fitness()
        # Sort the population based on fitness
        population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
        
        
        logger.info(f"Initial population: {len(population)} chromosomes")

        for i in range(max_iteration):

            self.log[i] = {"best_chromosome": population[0], "repair_time": 0, "crossover_time": 0, "mutation_time": 0, "fitness_time": 0, "total_time": 0, "elitism_time": 0, "selection_time": 0, "total_time": 0}

            logger.info(f"Iteration {i}")
            start = time.time()

            # Evolve the population
            
            population, elitism = self.__evolve_population(population,i)
            evolve_time = time.time() - start
            self.log[i]["total_time"] += evolve_time
            logger.info(f"Evolve time for iteration {i}: {evolve_time}")

            # Add the elitism back to the population
            population.add_chromosome(elitism)

            # Calculate the fitness of the population
            population.calculate_fitness()
            fitness_time = time.time() - start - evolve_time
            self.log[i]["fitness_time"] += fitness_time

            # Sort the population based on fitness
            population = Population(sorted(population, key=lambda chromosome: chromosome.fitness), population.fitness_manager)
        
            logger.info(f"Best chromosome after iteration {i}: {population[0].fitness}")
            logger.info(f"Worst chromosome after iteration {i}: {population[-1].fitness}")
            #line
            print("--------------------------------------------------")

            # gc.collect()
        return population[0]
    
    def configure(self, population_size: int, selection_manager: SelectionManager, crossover_manager: CrossoverManager, mutation_manager: MutationManager, repair_manager: RepairManager, elitism_size: int):
        self.population_size = population_size
        self.selection_manager = selection_manager
        self.crossover_manager = crossover_manager
        self.mutation_manager = mutation_manager
        self.repair_manager = repair_manager
        self.elitism_size = elitism_size

In [42]:
def generate_best_chromosome(max_iteration):
    factory = Factory()

    #fitness
    group_assignment_conflict_fitness = GroupAssignmentConflictFitness()
    group_assignment_conflict_fitness.configure(3, 1)

    assistant_distribution_fitness = AssistantDistributionFitness()
    assistant_distribution_fitness.configure(15, 50, 1, 1)

    fitness_manager = FitnessManager([group_assignment_conflict_fitness, assistant_distribution_fitness])

    population_size = 25

    selection_manager = SelectionManager([RouletteWheelSelection(), TournamentSelection(), ElitismSelection()])
    crossover_manager = CrossoverManager([SinglePointCrossover(), TwoPointCrossover(), UniformCrossover()])
    mutation_manager = MutationManager([SwapMutation(), ShiftMutation(), RandomMutation()])
    repair_manager = RepairManager([RepairTimeSlot()])
    elitism_size = 1
    elitism_selection = ElitismSelection()

    selection_manager.selection_probability = 0.75
    crossover_manager.crossover_probability = 0.75
    mutation_manager.mutation_probability = 0.75

    genetic_algorithm = GeneticAlgorithm(factory, population_size, fitness_manager, selection_manager, crossover_manager, mutation_manager, repair_manager, elitism_size, elitism_selection)

    return genetic_algorithm.run(max_iteration)

### Tabu Search
Algoritma pencarian lokal untuk disandingkan dengan algoritma utama, yaitu algoritma genetika. Algoritma ini berfungsi untuk memeperbaiki hasil dari algoritma genetika, sekaligus mengenalkan diversity pada hasil penjadwalan agar tidak terjebak pada local optimum.

Terdapat beberapa komponen yang perlu diperhatikan dalam algoritma ini, yaitu:
1. Tabu list, berfungsi untuk menyimpan solusi yang sudah pernah dijelajahi agar tidak dijelajahi lagi. Struktur data yang digunakan serupa dengan Populasi pada algoritma genetika, yaitu list of chromosome. Akan tetapi terdapat perbedaan properti, pada tabu list terdapat tabu tenure, yaitu jumlah iterasi yang harus dilewati sebelum solusi yang sudah pernah dijelajahi dapat dijelajahi lagi.
2. Aspiration criteria, berfungsi untuk memperbolehkan solusi yang sudah pernah dijelajahi untuk dijelajahi lagi, jika solusi tersebut lebih baik dari solusi terbaik yang pernah ditemukan.
3. Neighborhood structure, berfungsi untuk menghasilkan solusi yang berdekatan dengan solusi yang sedang dijelajahi.
4. Initial solution, berfungsi untuk menghasilkan solusi awal yang akan dijelajahi, pada kasus ini solusi awalnya adalah hasil dari algoritma genetika.
5. Strategy for selecting the next solution, berfungsi untuk memilih solusi yang akan dijelajahi selanjutnya. Terdapat beberapa strategi yang dapat digunakan, yaitu:
    - Best improvement, berfungsi untuk memilih solusi yang paling baik dari sekumpulan solusi yang dihasilkan oleh neighborhood structure.
    - First improvement, berfungsi untuk memilih solusi yang pertama kali ditemukan dari sekumpulan solusi yang dihasilkan oleh neighborhood structure.
    - Random, berfungsi untuk memilih solusi secara acak dari sekumpulan solusi yang dihasilkan oleh neighborhood structure.
6. Stopping criteria, berfungsi untuk menghentikan pencarian jika kriteria yang ditentukan telah terpenuhi. 
7. Tabu Search, algoritma utama yang menggabungkan semua komponen diatas.



#### Tabu List

In [43]:
#Tabu list class

from collections import deque

class TabuList:
    def __init__(self, max_size: int):
        self.max_size = max_size
        self.tabu_list = deque(maxlen=max_size)
    
    def __str__(self):
        return f"TabuList(tabu_list={self.tabu_list})"
    
    def __repr__(self):
        return self.__str__()
    
    def __contains__(self, chromosome: Chromosome):
        return chromosome in self.tabu_list
    
    def __len__(self):
        return len(self.tabu_list)
    
    def __getitem__(self, index):
        return self.tabu_list[index]
    
    def __setitem__(self, index, chromosome: Chromosome):
        self.tabu_list[index] = chromosome
    
    def __delitem__(self, index):
        del self.tabu_list[index]
    
    def __iter__(self):
        return iter(self.tabu_list)
    
    def __reversed__(self):
        return reversed(self.tabu_list)
    
    def __add__(self, chromosome: Chromosome):
        self.tabu_list.append(chromosome)

In [44]:
count = 0
for i in range(288):
    for j in range(288):
        if i != j:
            count += 1
print(count)

82656


In [75]:
#Neighborhood Class

from copy import deepcopy
from turtle import distance

class BaseNeighborhood:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Neighborhood(name={self.name})"
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self, chromosome: Chromosome):
        raise NotImplementedError("Neighborhood function not implemented")
    
class RandomSwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("SwapNeighborhood")
        self.neighborhood_percentage = 0.1
        self.swap_percentage = 0.1

    def __call__(self, chromosome: Chromosome) -> List[Chromosome]:
        '''Swap the time slot and the assistant for 10% of the genes and generate 10% of the neighbors'''
        neighbors = []
        for _ in range(int(len(chromosome) * self.neighborhood_percentage)):
            neighbor = deepcopy(chromosome)
            # Randomly swap 2 genes for 10% of the genes
            for _ in range(int(len(chromosome) * self.swap_percentage)):
                gene1 = random.choice(neighbor)
                gene2 = random.choice(neighbor)
                gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
                if gene1.laboratory == gene2.laboratory:
                    gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
            neighbors.append(neighbor)
        return neighbors
    
    def configure(self, neighborhood_percentage, swap_percentage):
        self.neighborhood_percentage = neighborhood_percentage
        self.swap_percentage = swap_percentage
        return self
    
class DistanceSwapNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("SwapNeighborhood")
        self.distance_percentage = 0.1
        self.distance_matrix = None
        
    def __call__(self, chromosome: Chromosome) -> List[Chromosome]:
        '''Swap genes based on furthest distance'''
        neighbors = []
        if self.distance_matrix is None:
            self.calculate_all_distance(chromosome)

        # Sort the distance from the furthest to the closest
        distance = sorted(self.distance_matrix, key=lambda distance: distance[2], reverse=True)

        # Select the top 10% of the distance
        selected_distance = distance[:int(len(distance) * self.distance_percentage)]

        print(selected_distance)


        # # Swap the genes
        # for distance in selected_distance:
        #     neighbor = deepcopy(chromosome)
        #     self.swap_gene(neighbor, distance[0], distance[1])
        #     neighbors.append(neighbor)

        # return neighbors
    
    def calculate_all_distance(self, chromosome: Chromosome):
        '''Calculate the distance between all genes'''
        self.distance_matrix = []
        for i in range(len(chromosome)):
            for j in range(i + 1, len(chromosome)):
                distance = self.calculate_distance(chromosome[i], chromosome[j])
                self.distance_matrix.append((i, j, distance))
    
    def calculate_distance(self, gene1: Gene, gene2: Gene):
        '''Calculate the distance between 2 genes'''
        # Calculate the distance between the time slot, time slot is TimeSlot tuple which contain date, day and shift
        time_slot_distance = self.calculate_time_slot_distance(gene1.time_slot, gene2.time_slot)
        if gene1.laboratory != gene2.laboratory:
            # If the laboratory is different, then the distance is 1
            return 1 + time_slot_distance
        # If the laboratory is the same, then calculate the distance between the assistant
        assistant_distance = self.calculate_assistant_distance(gene1.assistant, gene2.assistant)
        return assistant_distance + time_slot_distance
    
    def calculate_time_slot_distance(self, time_slot1: TimeSlot, time_slot2: TimeSlot):
        '''Calculate the distance between 2 time slots'''
        # Calculate the distance between the date
        date_distance = abs((time_slot1.date - time_slot2.date).days)
        # Calculate the distance between the day
        day_distance = abs(Constant.days.index(time_slot1.day) - Constant.days.index(time_slot2.day))
        # Calculate the distance between the shift
        shift_distance = abs(Constant.shifts.index(time_slot1.shift) - Constant.shifts.index(time_slot2.shift))
        return date_distance + day_distance + shift_distance
    
    def calculate_assistant_distance(self, assistant1: int, assistant2: int):
        '''Simple distance between 2 assistants, I'll change this later'''
        return abs(assistant1 - assistant2)
    
    def swap_gene(self, chromosome: Chromosome, index1: int, index2: int):
        '''Swap 2 genes in the chromosome'''
        if chromosome[index1].module != chromosome[index2].module:
            return
        chromosome[index1].time_slot, chromosome[index2].time_slot = chromosome[index2].time_slot, chromosome[index1].time_slot
        chromosome[index1].assistant, chromosome[index2].assistant = chromosome[index2].assistant, chromosome[index1].assistant
        
    def configure(self, distance_percentage):
        self.distance_percentage = distance_percentage
        return self

    
class ShiftNeighborhood(BaseNeighborhood):
    def __init__(self):
        super().__init__("ShiftNeighborhood")
        self.constant = Constant
    
    def __call__(self, chromosome: Chromosome) -> List[Chromosome]:
        
        neighbors = []
        for i in range(len(chromosome)):
            for j in range(len(chromosome)):
                if i != j:
                    neighbor = deepcopy(chromosome)
                    neighbor[i].time_slot = self.shift_time_slot(neighbor[i].time_slot)
                    neighbors.append(neighbor)
        return neighbors
    
    def shift_time_slot(self, time_slot: TimeSlot) -> TimeSlot:
        # Shift the time slot by 1 day
        if time_slot.day == "Saturday":
            return TimeSlot(time_slot.date + timedelta(days=2), "Monday", time_slot.shift)
        return TimeSlot(time_slot.date + timedelta(days=1), self.constant.days[self.constant.days.index(time_slot.day) + 1], time_slot.shift)

In [76]:
settings.DEBUG = False
best_chromosome = generate_best_chromosome(1)

INFO:__main__:Initial population: 25 chromosomes
INFO:__main__:Iteration 0
INFO:__main__:Evolve time for iteration 0: 0.6355493068695068
INFO:__main__:Best chromosome after iteration 0: 81
INFO:__main__:Worst chromosome after iteration 0: 107


--------------------------------------------------


In [77]:
#test swap neighborhood
chromosome = deepcopy(best_chromosome)
swap_neighborhood = DistanceSwapNeighborhood()

import cProfile
cp = cProfile.Profile()
cp.enable()
neighbors = swap_neighborhood(chromosome)
cp.disable()

[(0, 1, 5), (0, 2, 25), (0, 3, 13), (0, 4, 13), (0, 5, 6), (0, 6, 13), (0, 7, 24), (0, 8, 25), (0, 9, 17), (0, 10, 8), (0, 11, 23), (0, 12, 12), (0, 13, 2), (0, 14, 7), (0, 15, 13), (0, 16, 9), (0, 17, 28), (0, 18, 11), (0, 19, 3), (0, 20, 20), (0, 21, 10), (0, 22, 23), (0, 23, 20), (0, 24, 10), (0, 25, 22), (0, 26, 11), (0, 27, 25), (0, 28, 26), (0, 29, 9), (0, 30, 8), (0, 31, 7), (0, 32, 6), (0, 33, 24), (0, 34, 13), (0, 35, 24), (0, 36, 13), (0, 37, 11), (0, 38, 19), (0, 39, 15), (0, 40, 12), (0, 41, 17), (0, 42, 15), (0, 43, 11), (0, 44, 9), (0, 45, 24), (0, 46, 9), (0, 47, 20), (0, 48, 5), (0, 49, 9), (0, 50, 18), (0, 51, 10), (0, 52, 2), (0, 53, 9), (0, 54, 9), (0, 55, 6), (0, 56, 5), (0, 57, 10), (0, 58, 18), (0, 59, 10), (0, 60, 16), (0, 61, 27), (0, 62, 3), (0, 63, 11), (0, 64, 22), (0, 65, 11), (0, 66, 4), (0, 67, 13), (0, 68, 18), (0, 69, 10), (0, 70, 27), (0, 71, 13), (0, 72, 12), (0, 73, 18), (0, 74, 12), (0, 75, 18), (0, 76, 19), (0, 77, 10), (0, 78, 12), (0, 79, 10), (0,

In [68]:
cp.print_stats(sort='time')

         620967 function calls (620678 primitive calls) in 0.203 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    41328    0.065    0.000    0.097    0.000 1858482881.py:92(calculate_time_slot_distance)
        1    0.038    0.038    0.191    0.191 1858482881.py:73(calculate_all_distance)
    41328    0.031    0.000    0.139    0.000 1858482881.py:81(calculate_distance)
   165312    0.024    0.000    0.024    0.000 {method 'index' of 'list' objects}
   165312    0.010    0.000    0.010    0.000 {built-in method builtins.abs}
    82656    0.010    0.000    0.010    0.000 3979627171.py:16(__getitem__)
    41328    0.008    0.000    0.010    0.000 1858482881.py:102(calculate_assistant_distance)
        1    0.008    0.008    0.010    0.010 {built-in method builtins.sorted}
    41328    0.004    0.000    0.004    0.000 {method 'append' of 'list' objects}
    41328    0.002    0.000    0.002    0.000 1858482881.py:58(<lambda>